# Step 0 - Setup your environnement variables

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["UIFORM_API_KEY"] = "YOUR API KEY"

# Option 1 - Create a data structure from some examples

In [44]:
from uiform.client import UiForm

client = UiForm()

documents = ['visa/visa_a_miller.png','visa/visa_g_raymond.webp','visa/visa_m_demonceaux.webp']

generation_response = client.schemas.generate(
    documents=documents,
    model="gpt-4o-2024-11-20", 
    temperature=0
)

with open("visa/visa_schema_generated.json", "w") as f:
    json.dump(generation_response.json_schema, f, indent=2)

# Option 2 - Enhance an existing data structure from some examples

In [51]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import date
from uiform.client import UiForm

client = UiForm()

class CreditCardInfo(BaseModel):
    card_number: str 
    card_type: str
    cardholder_name: str
    expiration_date: str 
    card_network: str 
    additional_info: Optional[str]

promptify_response = client.schemas.promptify(
    raw_json_schema=CreditCardInfo.model_json_schema(), 
    documents=documents
)

with open("visa/visa_schema_promptified.json", "w") as f:
    json.dump(promptify_response.json_schema, f, indent=2)


# 3 - Extract informations from your documents

In [52]:
import json
from uiform.client import UiForm

with open("visa/visa_schema_promptified.json", "r") as f:
    json_schema = json.load(f)

document = "visa/visa_a_miller.png"

client = UiForm()
response = client.documents.extract(
    json_schema = json_schema,
    document = document,
    model="gpt-4o-mini-2024-07-18",
    temperature=0
)

In [55]:
print(json.dumps(response.model_dump(),indent=2))

{
  "id": "64baa15f-803e-4583-8ed5-2bbf4f3ea13f",
  "model": "gpt-4o-mini-2024-07-18",
  "seed": null,
  "temperature": 0.0,
  "document": {
    "id": "6d44400f962ffe7ea550db3892b84590fc764a9002ecdb4ad61481d42ed88b6a",
    "name": "visa_a_miller.png",
    "size": 216564,
    "mime_type": "image/png"
  },
  "extraction": {
    "card_number": "4000123456789010",
    "card_type": "Credit",
    "cardholder_name": "A. MILLER",
    "expiration_date": "12/20",
    "card_network": "Visa",
    "additional_info": null
  },
  "likelihoods": {
    "card_number": 0.9626727773451229,
    "card_type": 0.9998316352049432,
    "cardholder_name": 0.9889269278230699,
    "expiration_date": 0.9999998063873687,
    "card_network": 0.9988304590467317,
    "additional_info": 0.9999996871837189
  },
  "regex_instruction_results": null,
  "additional_context_message": "",
  "usage": {
    "model": "gpt-4o-mini-2024-07-18",
    "input_tokens": 877,
    "output_tokens": 43,
    "cost": 0.00015735
  },
  "modalit